In [1]:
import pyodbc
import pandas as pd
import numpy as np
import time
from texttransformation import StringTransform, RowTextTransform, TransformDataset
from sklearn import preprocessing

Alter dataset by injecting various random errors, typos etc. 

In [32]:
alteration_rules = [
    {
        'rule_Replace': ['none',''],
        'rule_RandomTypo': ['alpha', 2, 'replace'],
        'rule_ScrambleWords': [],
        'rule_DuplicateNumericSequence': [2],
        'rule_RemoveSpecialSymbols': [],
        'rule_RemoveStopWords': [],
        'rule_IncreaseWeightOfShortWords':[]
    },
    {
        'rule_Replace': ['none',''],
        'rule_RandomTypo': ['any', 2, 'add'],
        'rule_ScrambleWords': [],
        'rule_DuplicateNumericSequence': [4],
        'rule_RemoveSpecialSymbols': [],
        'rule_RemoveStopWords': [],
        'rule_IncreaseWeightOfShortWords':[]
    },
    {
        'rule_Replace': ['none',''],
        'rule_RandomTypo': ['digits', 1, 'add'],
        'rule_ScrambleWords': [],
        'rule_DuplicateNumericSequence': [3],
        'rule_RemoveSpecialSymbols': [],
        'rule_RemoveStopWords': [],
        'rule_IncreaseWeightOfShortWords':[]
    }
]

column_alteration_rules = []
for i in range(0, len(alteration_rules)):
    column_alteration_rules.append(
        {
            1: alteration_rules[i],
            2: alteration_rules[i],                    
            3: alteration_rules[i],
            4: alteration_rules[i],
            5: alteration_rules[i],
            6: alteration_rules[i],
            7: alteration_rules[i],
            8: alteration_rules[i],
            9: alteration_rules[i]
        })

result = []
for rule in range(0, len(alteration_rules)):
    transformer = TransformDataset(column_alteration_rules[rule])    
    result = result + transformer.execute(df_source.values.tolist())

df_source_alterd  = pd.DataFrame(result)